In [ ]:
import os
import pandas as pd
import numpy as np

path_to_aggregated_data = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Auxillary Data/NOAA Disaster Data/final_weather.csv"

news_weather_df = pd.read_csv(path_to_aggregated_data)
news_weather_df.describe()

print(len(news_weather_df))

In [ ]:
# first we are going to ignore anything that got zero mentions as that will murder our data and analysis
news_weather_df = news_weather_df[news_weather_df["mentions"] > 0]

# now it is time to do some modeling -- the different types of event that we can have occur are
print(len(news_weather_df))
print(news_weather_df.columns)

In [ ]:
# we are going to append state data to our disasters file

ignore = "Multiple states"
path_to_aggregated_data = path_to_aggregated_data
path_to_gpt = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/tristan_analysis/gpt_states_affected.txt"
curr_event = None
states_affected = ""

info_dict = {}

news_weather_df["states_affected"] = ""

with open(path_to_gpt, "r") as states_file:

    lines = states_file.readlines()

    lines = [item.strip() for item in lines if item.strip() != ""]

    for l in lines:

        # if there is a colon in the line, then this is the start of new_data
        if ":" in l:

            print(curr_event)
            print(states_affected)

            if curr_event is not None:

                info_dict[curr_event] = states_affected

            curr_event = l[:-1]
            states_affected = ""
            continue
            
        elif ignore in l:
            continue
        
        # otherwise this is a state that we want to analyze
        try:
            
            segments = l.split(" - ")
            
            # first is state
            states_affected += segments[0].strip()
            states_affected += "_"

            # second is score
            if "(" in segments[1]:
                states_affected += segments[1][ : segments[1].index("(")].strip()

            else:
                states_affected += segments[1].strip()

            states_affected += "|"



        except:

            continue

news_weather_df["states_affected"] = news_weather_df.apply(lambda row: info_dict.get(row['name'], row["states_affected"]), axis=1)


In [ ]:
path_to_aggregated_data = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Auxillary Data/NOAA Disaster Data/final_weather_states.csv"

news_weather_df.dropna(subset=["states_affected"], inplace=True)
news_weather_df = news_weather_df[news_weather_df["states_affected"] != ""]

news_weather_df.to_csv(path_to_aggregated_data, index=False)

In [ ]:
# transform to a state, severity mapping
state_codes = {
        "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
        "California": "CA", "Colorado": "CO", "Connecticut": "CT",
        "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
        "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
        "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME",
        "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI",
        "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO",
        "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
        "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
        "New York": "NY", "North Carolina": "NC", "North Dakota": "ND",
        "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA",
        "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD",
        "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
        "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
        "Wisconsin": "WI", "Wyoming": "WY"
    }

def split_state_severity(string):

    pairs = string.split("|")
    pairs = [pairing.split("_") for pairing in pairs if pairing != '']

    ret_p = []

    for p in pairs:
        
        try :

            ret_p.append([state_codes.get(p[0], "Unknown"), p[1]])

        except:

            print(p)

    return ret_p

news_weather_df["code_severity"] = news_weather_df["states_affected"].apply(split_state_severity)

In [ ]:
news_weather_df["code_severity"]

In [ ]:
type_disaster_dict = {}

for t in news_weather_df["disaster_type"].unique():

    print(t)
    type_disaster_dict[t] = news_weather_df[news_weather_df["disaster_type"] == t]

In [ ]:
import matplotlib.pyplot as plt
import datetime

path_to_market_id_mapping = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/crucial_data_and_files/code_to_region_state_mapping.csv"
market_id_mapping_df = pd.read_csv(path_to_market_id_mapping)

path_to_shutdown_data = "/Users/tristanbrigham/Desktop/basic_cancelled_final copy.csv"
shutdown_df = pd.read_csv(path_to_shutdown_data)
shutdown_df = shutdown_df[shutdown_df["amt_quarters_past"] >= 4]

markets_amt_ppl = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/crucial_data_and_files/markets_to_people_per_day.csv"
markets_amt_ppl_df = pd.read_csv(markets_amt_ppl)


# the amount of quarters after the initial disaster that we want to check for airport closures
max_q_count = 4

# max amt passengers for small airport
max_pass_small = 250
max_pass_route = 100

def get_airport_shutdowns_by_state_codes(name, date_start, date_end, first_quarter, code_sev_pairings):

    affected_market_ids_df = pd.DataFrame(columns=market_id_mapping_df.columns)

    # getting all affected market ids
    for pair in code_sev_pairings:

        affected_market_ids_df = pd.concat([affected_market_ids_df, market_id_mapping_df[market_id_mapping_df["state"] == pair[0]]])


    # now go through the data file that has the route shutdowns for 2 years after and add to a dataframe
    c_year = date_start.year
    c_quar = first_quarter
    
    
    small_airports_total = 0

    for market_id in affected_market_ids_df["market_id"]:

        small_airports_total += get_amt_small_airports(market_id, c_year, c_quar)


    total_shutdowns_state_quarter = pd.DataFrame(columns = shutdown_df.columns)

    q_count = 0

    # go through each of the quarters and check
    while q_count < max_q_count:

        # print(f"Looking at {c_year} : {c_quar}")

        # get all of the shutdowns that happen in those markets in that quarter
        this_q_shutdowns_mid = shutdown_df[(shutdown_df["passengers"] < max_pass_route) & (shutdown_df["Year"] == c_year) & (shutdown_df["quarter"] == c_quar) & ((shutdown_df['citymarketid_1'].isin(affected_market_ids_df['market_id'])) | (shutdown_df['citymarketid_2'].isin(affected_market_ids_df['market_id'])))]

        # cat the market_ids
        total_shutdowns_state_quarter = pd.concat([total_shutdowns_state_quarter, this_q_shutdowns_mid])

        # increase index
        c_quar += 1

        if c_quar > 4:

            c_quar = 0
            c_year += 1

        q_count += 1

    return name, total_shutdowns_state_quarter, small_airports_total, len(affected_market_ids_df)

        
def get_first_quarter(q1, q2, q3, q4):

    if q1 == True:
        return 1

    elif q2 == True:
        return 2

    elif q3 == True:
        return 3

    else:
        return 4
        

def get_amt_small_airports(code, year, quarter):

    return len(markets_amt_ppl_df[(markets_amt_ppl_df["Year"] == year) & (markets_amt_ppl_df["quarter"] == quarter) & (markets_amt_ppl_df["citymarketid"] == code)  & (markets_amt_ppl_df["passengers"] < max_pass_small)])

        

In [ ]:
# let's try to figure out if there is a correlation between large weather event and airports closing
news_weather_df['start'] = pd.to_datetime(news_weather_df['start'])
news_weather_df['end'] = pd.to_datetime(news_weather_df['end'])

news_weather_df["first_quarter"] = news_weather_df.apply(lambda row: get_first_quarter(row["Q1"], row["Q2"], row["Q3"], row["Q4"]), axis=1)

# news_weather_df
date_str = "1/1/1997"
date_start = datetime.datetime.strptime(date_str, "%m/%d/%Y")
date_str = "1/1/2020"
date_end = datetime.datetime.strptime(date_str, "%m/%d/%Y")

news_weather_df['year'] = pd.to_datetime(news_weather_df['start']).dt.year.astype(int)

after_1997_weather_df = news_weather_df[(news_weather_df['start'] > date_start) & (news_weather_df['start'] < date_end)]

weather_impact_exposure = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}

weather_impact_normalized = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}

weather_mention_exposure = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}

weather_mention_normalized = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}

weather_mention_ratio_exposure = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}

weather_mention_ratio_normalized = {
    "Freeze" : [],
    "Severe Storm" : [],
    "Tropical Cyclone" : [],
    "Wildfire" : [],
    "Winter Storm" : [],
    "Flooding" : [],
    "Drought" : [],
}


avg_dict = {
    "Freeze" : [0, 0],
    "Severe Storm" : [0, 0],
    "Tropical Cyclone" : [0, 0],
    "Wildfire" : [0, 0],
    "Winter Storm" : [0, 0],
    "Flooding" : [0, 0],
    "Drought" : [0, 0],
}

# total_shutdowns_after_disaster_locality 
output_applied = after_1997_weather_df.apply(lambda row: get_airport_shutdowns_by_state_codes(row['name'], row['start'], row['end'], row["first_quarter"], row["code_severity"]), axis=1)

In [ ]:

for disaster_name, disaster_shutdowns, small_airports, total_airports in output_applied:

    # init print
    # print(f"Disaster Name: {disaster_name}")
    # print(f"Disaster type: {news_weather_df[news_weather_df['name'] == disaster_name]['disaster_type'].values[0]}")
    # print(f"Amt of shutdowns (region): {len(disaster_shutdowns)}")
    # print(f"Amt of small airports (region): {small_airports}")
    # print(f"Cost of disaster: {news_weather_df[news_weather_df['name'] == disaster_name]['cost'].values[0]}")
    # print(f"Ratio cost to airports: {len(disaster_shutdowns) / news_weather_df[news_weather_df['name'] == disaster_name]['cost'].values[0]}")
    # print()

    if (len(disaster_shutdowns) + 1) / (small_airports + 1) < 0.25:
        print(f"Disaster Name: {disaster_name}")
        print((len(disaster_shutdowns) + 1) / (small_airports + 1))

    # print(2 * int(news_weather_df[news_weather_df["name"] == disaster_name]['year'].iloc[0]))

    # creating impact dict
    weather_impact_exposure[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (small_airports + 1) , news_weather_df[news_weather_df["name"] == disaster_name]['cost'].values[0], ])
    weather_impact_normalized[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (total_airports + 1) , news_weather_df[news_weather_df["name"] == disaster_name]['cost'].values[0], ])

    # mentions vs. shutters
    weather_mention_exposure[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (small_airports + 1) , news_weather_df[news_weather_df["name"] == disaster_name]['mentions'].values[0] * (1.25 ** (abs(2020 - int(news_weather_df[news_weather_df["name"] == disaster_name]['start'].iloc[0].year))))])
    weather_mention_normalized[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (total_airports + 1) , news_weather_df[news_weather_df["name"] == disaster_name]['mentions'].values[0] * (1.25 ** (abs(2020 - int(news_weather_df[news_weather_df["name"] == disaster_name]['start'].iloc[0].year))))])
    
    # mention-cost relationship vs. shutters
    weather_mention_ratio_exposure[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (small_airports + 1) ,  news_weather_df[news_weather_df["name"] == disaster_name]['mentions'].values[0] * (1.25 ** (abs(2020 - int(news_weather_df[news_weather_df["name"] == disaster_name]['start'].iloc[0].year)))) / news_weather_df[news_weather_df["name"] == disaster_name]['cost'].values[0]])
    weather_mention_ratio_normalized[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]].append([(len(disaster_shutdowns) + 1) / (total_airports + 1) ,  news_weather_df[news_weather_df["name"] == disaster_name]['mentions'].values[0] * (1.25 ** (abs(2020 - int(news_weather_df[news_weather_df["name"] == disaster_name]['start'].iloc[0].year)))) / news_weather_df[news_weather_df["name"] == disaster_name]['cost'].values[0]])

    # updating the averages:
    avg_dict[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]] = [avg_dict[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]][0] + news_weather_df[news_weather_df["name"] == disaster_name]['cost'].values[0], avg_dict[news_weather_df[news_weather_df["name"] == disaster_name]['disaster_type'].values[0]][1] + len(disaster_shutdowns)]

    # figuring out the total amount of route removals to disaster

In [ ]:
print(avg_dict.items())

for key, item in avg_dict.items():

    print(f"{key} : Cost: {item[0]} / Cancellations: {item[1]}")
    print(f"{item[1] / item[0]}")
    print()

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_impact_exposure[t])

    x = [point[1] for point in weather_impact_exposure[t]]
    y = [point[0] for point in weather_impact_exposure[t]]
    
    # # if t == "Severe Storm":
    # #     x = [point[1] for point in weather_impact_exposure[t] if point[1] < 8000]
    # #     y = [point[0] for point in weather_impact_exposure[t] if point[1] < 8000]

    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')

    
    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)
    print(f'Correlation {t} - Percent Small Airports Shuttered to Cost: {correlation_coefficient:.2f}')

    
    # # Set the title for the graph
    # plt.title(t)

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Cost of Disaster')

    # # Show the plot
    # plt.show()

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_impact_normalized[t])

    x = [point[1] for point in weather_impact_normalized[t]]
    y = [point[0] for point in weather_impact_normalized[t]]
    
    # # if t == "Severe Storm":
    # #     x = [point[1] for point in weather_impact_exposure[t] if point[1] < 8000]
    # #     y = [point[0] for point in weather_impact_exposure[t] if point[1] < 8000]

    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')

    
    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)
    print(f'Correlation {t} - Percent Total Airports Shuttered to Cost: {correlation_coefficient:.2f}')

    
    # # Set the title for the graph
    # plt.title(t)

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Cost of Disaster')

    # # Show the plot
    # plt.show()

In [ ]:
# for i in range(10):

#     max_q_count = i

#     # total_shutdowns_after_disaster_locality 
#     output_applied = after_1997_weather_df.apply(lambda row: get_airport_shutdowns_by_state_codes(row['name'], row['start'], row['end'], row["first_quarter"], row["code_severity"]), axis=1)

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_mention_normalized[t])

    x = [point[1] for point in weather_mention_normalized[t]]
    y = [point[0] for point in weather_mention_normalized[t]]
    
    
    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)
    print(f'Correlation {t} - Percent Total Airports Shuttered to Media Mentions: {correlation_coefficient:.2f}')


    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')


    # # Set the title for the graph
    # plt.title(f"Mentions vs. Shutdown: {t}")

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Mentions')

    # # Show the plot
    # plt.show()

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_mention_exposure[t])

    x = [point[1] for point in weather_mention_exposure[t]]
    y = [point[0] for point in weather_mention_exposure[t]]
    
    
    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)
    print(f'Correlation {t} - Percent Small Airports Shuttered to Media Mentions: {correlation_coefficient:.2f}')


    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')


    # # Set the title for the graph
    # plt.title(f"Mentions vs. Shutdown: {t}")

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Mentions')

    # # Show the plot
    # plt.show()

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_mention_ratio_exposure[t])

    x = [point[1] for point in weather_mention_ratio_exposure[t]]
    y = [point[0] for point in weather_mention_ratio_exposure[t]]
    
    
    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    print(f'Correlation {t} - Percent Small Airports Shuttered to Media Mentions/Cost Ratio: {correlation_coefficient:.2f}')
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)


    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')


    # # Set the title for the graph
    # plt.title(f"Mentions vs. Shutdown: {t}")

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Mentions')

    # # Show the plot
    # plt.show()

In [ ]:
# go through each of the disaster categories and plot the cost of the disaster versus the amount of airports that shut down
for t in news_weather_df["disaster_type"].unique():

    # print(weather_mention_ratio_normalized[t])

    x = [point[1] for point in weather_mention_ratio_normalized[t]]
    y = [point[0] for point in weather_mention_ratio_normalized[t]]
    

    correlation_coefficient = np.corrcoef(x, y)[0, 1]
    print(f'Correlation {t} - Percent Total Airports Shuttered to Media Mentions/Cost Ratio: {correlation_coefficient:.2f}')
    # plt.text(0.7, 0.9, f'Correlation: {correlation_coefficient:.2f}', ha='center', va='center', transform=plt.gca().transAxes)

    # plt.scatter(x, y)

    # # Perform linear regression to fit a line to the data
    # coefficients = np.polyfit(x, y, 1)  # The number 1 represents linear regression (fitting a straight line)

    # # Create a linear regression model based on the obtained coefficients
    # linear_model = np.poly1d(coefficients)

    # # Generate the y values of the fitted line
    # y_fit = linear_model(x)

    # # Plot the fitted line
    # plt.plot(x, y_fit, color='red', label='Fitted Line')


    # # Set the title for the graph
    # plt.title(f"Mentions vs. Shutdown: {t}")

    # # Set labels for x and y axes
    # plt.ylabel('Airports Shut Down')
    # plt.xlabel('Mentions')

    # # Show the plot
    # plt.show()